# Exercise 1.2 – Car Counting (L→R then Up)
This notebook counts the number of cars that follow a left-to-right then upward path in `Traffic_Laramie_2.mp4` using background subtraction and tracking.

In [1]:
!pip install opencv-python-headless Pillow numpy

In [2]:
import cv2
import numpy as np
from IPython.display import display, clear_output
import PIL.Image
import time


## Parameters
Constants for region of interest, detection, tracking, and path logic.

In [3]:
VIDEO = "Traffic_Laramie_1.mp4"

ROI_FRAC = (0.45, 0.90, 0.10, 0.90)
N_BASELINE = 90
BLUR_K = 21
DIFF_THRESH = 30
MIN_AREA = 1700
ALPHA_SMOOTH = 0.92
WRITE_OUT = True

VLINE_FRAC = 0.45
HLINE_FRAC = 0.55
ORDER_WINDOW = 290
MIN_SPEED = 0.4

IOU_MIN = 0.10
DIST_MAX = 80
DIST_MAX_STATE1 = 160
MISS_MAX = 8
MISS_MAX_STATE1 = 24


## Helper Functions

In [4]:
def bbox_from_cxcywh(cx, cy, w, h):
    x1 = cx - w/2; y1 = cy - h/2
    x2 = x1 + w;  y2 = y1 + h
    return (x1, y1, x2, y2)

def iou(boxA, boxB):
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB
    inter_x1 = max(ax1, bx1); inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2); inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1); ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    areaA = (ax2-ax1)*(ay2-ay1); areaB = (bx2-bx1)*(by2-by1)
    return inter / (areaA + areaB - inter + 1e-6)

def extract_roi(frame):
    h, w = frame.shape[:2]
    y1, y2, x1, x2 = ROI_FRAC
    Y1, Y2, X1, X2 = int(y1*h), int(y2*h), int(x1*w), int(x2*w)
    return frame[Y1:Y2, X1:X2], (Y1, X1)

def to_gray_blur(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(g, (BLUR_K, BLUR_K), 0)


## Build Background Model

In [5]:
cap = cv2.VideoCapture(VIDEO)
if not cap.isOpened():
    raise RuntimeError(f"Could not open {VIDEO}")

gray_stack = []
while len(gray_stack) < N_BASELINE:
    ok, f = cap.read()
    if not ok:
        break
    roi, _ = extract_roi(f)
    gray_stack.append(to_gray_blur(roi))

if not gray_stack:
    raise RuntimeError("No frames read to build baseline.")

baseline = np.median(np.stack(gray_stack, axis=0), axis=0).astype(np.uint8)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

fps = cap.get(cv2.CAP_PROP_FPS) or 25
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

if WRITE_OUT:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(VIDEO.replace(".mp4","_counted.mp4"), fourcc, fps, (w, h))


## Tracking and Counting Loop

In [6]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
tracks = []
total_count = 0
frame_idx = 0
next_id = 0

while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame_idx += 1
    roi, (offY, offX) = extract_roi(frame)
    g = to_gray_blur(roi)

    delta = cv2.absdiff(g, baseline)
    _, mask = cv2.threshold(delta, DIFF_THRESH, 255, cv2.THRESH_BINARY)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    dets = []
    for c in cnts:
        if cv2.contourArea(c) < MIN_AREA: continue
        x, y, w2, h2 = cv2.boundingRect(c)
        cx, cy = x + w2/2, y + h2/2
        dets.append({'cx':cx,'cy':cy,'w':w2,'h':h2,'box':(x, y, x+w2, y+h2)})

    # Matching and track management omitted here for brevity
    # Use same logic as full script for track matching and state transitions

    roi_h, roi_w = roi.shape[0], roi.shape[1]
    vx = int(VLINE_FRAC * roi_w)
    hy = int(HLINE_FRAC * roi_h)

    cv2.line(roi, (vx, 0), (vx, roi_h-1), (0,165,255), 2)
    cv2.line(roi, (0, hy), (roi_w-1, hy), (0, 0, 255), 2)

    frame[offY:offY+roi.shape[0], offX:offX+roi.shape[1]] = roi

    elapsed_s = frame_idx / fps
    cpm = total_count / (elapsed_s/60) if elapsed_s > 0 else 0.0
    cv2.putText(frame, f"Count: {total_count}", (20, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
    cv2.putText(frame, f"Cars/min: {cpm:.2f}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)

    # GUI-safe display
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = PIL.Image.fromarray(img_rgb)
    clear_output(wait=True)
    display(img_pil)
    if WRITE_OUT:
        out.write(frame)
    time.sleep(0.001)

    # if frame_idx > 300:  # Optional stop
    #     break

cap.release()
if WRITE_OUT: out.release()
print(f"Done. Total cars: {total_count}")


KeyboardInterrupt: 